In [48]:
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd

def calculate_llt(prices, alpha=0.05):
    # 转换为numpy数组以避免pandas索引问题
    if hasattr(prices, 'values'):
        prices = prices.values
    
    n = len(prices)
    llt = np.zeros(n)
    if n >= 1:
        llt[0] = prices[0]
    if n >= 2:
        llt[1] = prices[1]
    
    a1 = alpha - (alpha**2) / 4
    a2 = (alpha**2) / 2
    a3 = alpha - 3 * (alpha**2) / 4
    a4 = 2 * (1 - alpha)
    a5 = - (1 - alpha)**2
    
    for t in range(2, n):
        llt[t] = a1 * prices[t] + a2 * prices[t-1] - a3 * prices[t-2] + a4 * llt[t-1] + a5 * llt[t-2]
    
    return llt

def llt_slope_signal(df, d: int=30, slope_window=5, thresh=(-0.02, 0.02)):
    df = df.copy()
    alpha = 2 / (d + 1)
    df["llt"] = calculate_llt(df["close"], alpha)
    df['slope'] = (df["llt"].rolling(slope_window)
                    .apply(lambda x: np.polyfit(np.arange(slope_window), x, 1)[0])
                 )

    signals = pd.Series(0, index=df.index)
    signals[df['slope'] >= thresh[1]] = 1
    signals[df['slope'] <= thresh[-1]] = -1
    signals.ffill(inplace = True)
    
    return signals

class LLTSignal(bt.Indicator):
    lines = ("signal",)
    params = (("d", 60), ("slope_window", 5), ("thresh", (-0.02, 0.02)))
    
    def __init__(self):
        self.signals = llt_slope_signal(self.data._dataname, self.p.d, self.p.slope_window, self.p.thresh)
        self.signal_index = 0
    
    def next(self):
        if self.signal_index < len(self.signals):
            self.lines.signal[0] = self.signals.iloc[self.signal_index]
            self.signal_index += 1
        else:
            self.lines.signal[0] = 0

def get_price(symbol, start_date, end_date):
    pro = pro_api()

    price_df = pro.index_daily(
        ts_code=symbol,
        start_date=start_date.strftime("%Y%m%d"),
        end_date=end_date.strftime("%Y%m%d"),
    )

    price_df = (
        price_df.rename({"trade_date": "date", "ts_code": "asset"}, axis=1)
        .sort_values("date", ascending=True)
        .set_index("date")
    )

    return price_df

start = datetime(2013, 1, 1)
end = datetime(2024, 12, 31)

prices = get_price("000001.SH", start, end)
prices.index = pd.to_datetime(prices.index)

cerebro = bt.Cerebro()

cerebro.broker.setcash(1_000_000)
cerebro.broker.setcommission(commission=0.0001)  # 0.1%手续费
cerebro.addsizer(bt.sizers.PercentSizer, percents=95)

data = btfeeds.PandasData(dataname=prices)
cerebro.adddata(data)
cerebro.add_signal(bt.SIGNAL_LONGSHORT, LLTSignal)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

cerebro.addwriter(bt.WriterFile, rounding=2, csv=True, out="/tmp/test_signal.csv")

results = cerebro.run()
strat = results[0]

returns = strat.analyzers.returns.get_analysis()
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
print(f"总收益率: {returns['rtot']:.2%}")
print(f"年化收益率: {returns['rnorm']:.2%}")
print(f"夏普比率: {sharpe['sharperatio']:.2f}")
print(f"最大回撤: {drawdown['max']['drawdown']:.2f}%")


总收益率: 63.29%
年化收益率: 5.63%
夏普比率: 0.29
最大回撤: 37.04%
